In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
df1 = pd.read_csv('../input/banfakenews/Authentic-48K.csv', na_values=['#NAME?'])
df2 = pd.read_csv('../input/banfakenews/Fake-1K.csv', na_values=['#NAME?'])
df3 = pd.read_csv('../input/banfakenews/LabeledAuthentic-7K.csv', na_values=['#NAME?'])
df4 = pd.read_csv('../input/banfakenews/LabeledFake-1K.csv', na_values=['#NAME?'])

In [ ]:
df1.shape

In [ ]:
df2.shape

In [ ]:
df3.shape

In [ ]:
df4.shape

In [ ]:
df = pd.concat([df1, df2, df3, df4])

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.drop('date', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
# How much data is missing
df.isnull().sum().sort_values(ascending=False).head()

In [ ]:
df.drop(['relation','F-type'],axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df['source'].replace('NaN','Not Found',inplace=True)

In [ ]:
df.head()

In [ ]:
df = df.fillna({'source':'Not Found'})
df

In [ ]:
# How much data is missing
df.isnull().sum().sort_values(ascending=False).head()

In [ ]:
# Categorical values or continuous
df.dtypes.sort_values().to_frame('feature_type').groupby(by = 'feature_type').size().to_frame('count').reset_index()

In [ ]:
print(df['category'].value_counts())

In [ ]:
df.drop_duplicates(inplace= True)

In [ ]:
x = df.category
y = df.source

In [ ]:
category_dummies = pd.get_dummies(x, prefix='category', columns=['category'])

In [ ]:
df = pd.concat([df,category_dummies], axis=1)

In [ ]:
df.head()

In [ ]:
print(df['source'].value_counts())

In [ ]:
source_dummies = pd.get_dummies(y, prefix='source', columns=['source'])

In [ ]:
df = pd.concat([df,source_dummies], axis=1)

In [ ]:
df.head()

In [ ]:
df.drop(['category','source'],axis=1, inplace=True)

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
h = df.headline

In [ ]:
h.head(5)

In [ ]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
h.columns = ['headline']
h = h.sort_index()
h

In [ ]:
h.drop_duplicates(inplace= True)
h

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(h.apply(round1))
data_clean

In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.headline.apply(round2))
data_clean

In [ ]:
# We are going to create a document-term matrix using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer


cv = CountVectorizer(tokenizer=lambda x: x.split())
data_cv = cv.fit_transform(data_clean.headline)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

In [ ]:
c = df.content

In [ ]:
c.head()

In [ ]:
c.columns = ['headline']
c = c.sort_index()
c

In [ ]:
c.drop_duplicates(inplace= True)
c

In [ ]:
# Let's take a look at the updated text
data_clean_content = pd.DataFrame(c.apply(round1))
data_clean_content

In [ ]:
# Let's take a look at the updated text
data_clean_content = pd.DataFrame(data_clean_content.content.apply(round2))
data_clean_content

In [ ]:
# We are going to create a document-term matrix using CountVectorizer

data_cv_2 = cv.fit_transform(data_clean_content.content)
data_dtm_2 = pd.DataFrame(data_cv_2.toarray(), columns=cv.get_feature_names())
data_dtm_2.index = data_clean.index
data_dtm_2